In [1]:
 import ee
import time
ee.Initialize()

In [2]:
geometry2 = ee.Geometry.Polygon(
  [[[75.59985792881425, 14.526591885743592],
    [75.59991139162679, 14.526596345918215],
    [75.59987125011487, 14.526600779378652],
    [75.59985792881425, 14.526591885743592]]]
);
AOI = geometry2

In [3]:
START_DATE = ee.Date('2019-01-22')
END_DATE = START_DATE.advance(1,'month')

CLOUD_FILTER = 100
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50
aoi = AOI.buffer(6400).bounds()

In [4]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    """ Import and filter S2 SR. Import and filter s2cloudless.
    Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    Import and filter S2 SR """
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                 .filterBounds(aoi)
                 .filterDate(start_date, end_date)
                 .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
                        .filterBounds(aoi)
                        .filterDate(start_date, end_date))
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [5]:
def add_cloud_bands(img):
    """ Get s2cloudless image, subset the probability band.
    Condition s2cloudless by the probability threshold value.
    Add the cloud probability layer and cloud mask as image bands. """
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [6]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)
    # not_water = img.normalizedDifference(['B3', 'B8']).lt(0.2) # Use this if you are using the TOA version
    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')
    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))
    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST * 10)
                    .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
                    .select('distance')
                    .mask()
                    .rename('cloud_transform'))
    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [7]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)
    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER * 2 / 20)
                       .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
                       .rename('cloudmask'))
    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [8]:
def addNDVI(image):
    return image.addBands(image.normalizedDifference(['B8', 'B4']))

In [9]:
# ==============================MIN and MAX cloudy image========================
s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE).sort('CLOUD_COVERAGE_ASSESSMENT',True)
s2_cloud_image_collection = s2_sr_cld_col_eval.map(add_cld_shdw_mask).map(addNDVI)
image_ids = s2_cloud_image_collection.aggregate_array('system:id')
image_ids_list = image_ids.getInfo()
#print(image_ids_list)
# print(image_ids_list[0])
# print(image_ids_list[-1])

In [10]:
# get 1st image in collection which is the min cloud image
cloud_free_image = s2_cloud_image_collection.filter(ee.Filter.eq('system:id', image_ids_list[0])).first()
cloud_free_image_date = ee.Date(cloud_free_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

#get laste image in the collection which is the max cloud image
cloudy_image = s2_cloud_image_collection.filter(ee.Filter.eq('system:id', image_ids_list[-1])).first()
cloudy_image_date = ee.Date(cloudy_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

print(cloud_free_image_date,'min_cloudy_image_date')
print(cloudy_image_date,'max_cloudy_image_date')

2019-02-16 min_cloudy_image_date
2019-01-27 max_cloudy_image_date


In [11]:
# Get corresponding S1 images
s1_start_date_cloudy = ee.Date(cloudy_image_date)
s1_end_date_cloudy = s1_start_date_cloudy.advance(12,'day')

s1_start_date_cloud_free = ee.Date(cloud_free_image_date)
s1_end_date_cloud_free = s1_start_date_cloud_free.advance(12,'day')

vh = ee.ImageCollection('COPERNICUS/S1_GRD')\
.filterDate(s1_start_date_cloud_free,s1_end_date_cloud_free) \
.filterBounds(aoi) \
.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
.filter(ee.Filter.eq('instrumentMode', 'IW')) \
.max() \
.clip(aoi) \
.select(['VH','VV'])

In [12]:
select_bands = ['B2','B3','B4','B5','B8','nd','clouds','shadows','VH','VV' ]
img = cloud_free_image.clip(aoi)
stacked_image = img.addBands(vh)
stacked_image = stacked_image.select(select_bands).reproject(**{'crs': 'EPSG:4326', 'scale': 10}).toFloat()

In [13]:
# provide relavent g-drive folder name and task name
folder_name = 'OptiSAR_Images'
description = 'OptiSAR_Images'

In [14]:
task_config = {
    'image': stacked_image,
    'description': description,
    'folder': folder_name,
    'scale': 10,
    'fileDimensions': 256,
    'region': aoi,
    'crs': 'EPSG:4326',
}

In [15]:
task = ee.batch.Export.image.toDrive(**task_config)
task.start()
print('started export for ' + task_config['description'])
# evaluate task every minute for n minutes
for n in range(1, 300):
    print('running for ' + str(n) + ' minutes')
    time.sleep(60)
    status = task.status()
    # if still running, do another loop
    if status['state'] == 'RUNNING':
        pass
    # if completed, do another loop
    elif status['state'] == 'COMPLETED':
        print('extraction completed...sleeping 10 sec')
        time.sleep(10)
        break
    elif status['state'] == 'FAILED':
        print('Extraction failed with status: ' + status['error_message'])
        break
    else:
        print("Extraction ran out of time, probably something went wrong.")
        print(status['state'])
        pass


started export for OptiSAR_Images
running for 1 minutes
running for 2 minutes
extraction completed...sleeping 10 sec
